In [1]:
from src.experimental_config import *
from src.policies import *
from src.simulator import MarketSimulator, MultiSimRunner, DIRECTIONAL_TRADING, NAIVE, RIGID, DIRECTIONAL_INCENTIVE_TRADING, COL_GEN, PRUNED_DIRECTIONAL_TRADING
from src.forecasting import *

import os
import pickle
from time import time

# delete all files in the experiments folder
for filename in os.listdir('./experiments'):
    file_path = os.path.join('./experiments', filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


generate_experiments(
    price_data_dir = './raw_data/spx_stock_prices.parquet',
    covariates = None,
    num_experiments = 100,
    output_dir = './experiments',
    min_horizon = 30,
    max_horizon = 90,
    max_num_stocks = 40,
    min_num_stocks = 10,
    min_tx_cost = 2,
    max_tx_cost = 10,
    max_budget = 350000,
    min_budget = 10000,
    forecast_model='GBM',
    forecast_params = {'var_multiplier': 20}
)

100%|██████████| 100/100 [00:24<00:00,  4.15it/s]


In [2]:
sim_runner = MultiSimRunner(
    experiments_directory='./experiments',
    policies = [NAIVE, DIRECTIONAL_TRADING, PRUNED_DIRECTIONAL_TRADING]
)
sim_runner.run()
sim_runner.results.to_csv('./results.csv')

Running Naive on 20180405_78_18_7_319924:   1%|          | 1/100 [08:18<13:42:56, 498.75s/it]                     


KeyboardInterrupt: 

In [ ]:
# import pickle
# from time import time
# # from tqdm.notebook import tqdm
# results = []
# # get the first file in the experiments folder
# # pbar = tqdm(os.listdir('./experiments'))
# for filename in os.listdir('./experiments'):
#     try:
#         file_path = os.path.join('./experiments', filename)
#         with open(file_path, "rb") as f:
#             exp = pickle.load(f)
#         # display the experiment id in the progress bar
#         print(f"Running {exp.exp_id}")
#         t1 = time()
#         policy = DirectionalTradingPolicy(exp, verbose=False)
#         simulator = MarketSimulator(exp, policy, verbose=False)

#         result = simulator.run()
#         t2 = time()
#         results.append({
#             "solve_time": np.mean(simulator.solve_times),
#             "forecast_time": np.mean(simulator.forecast_times),
#             "total_trades": simulator.total_trades,
#             "feasibility": simulator.status,
#             "horizon": exp.horizon
#         })
#         del exp
#         del policy
#         del simulator 
#         gc.collect()
#     except Exception as e:
#         print(f"\tFailed to run {exp.exp_id}. Reason: {e}")
#         continue